In [ ]:
import math
import numpy as np
import pandas as pd
from typing import Tuple, List, Optional

# MA7010 Assignment 1: Primes and Prime Testing


In [ ]:
def gcd_recursion(a: int, b: int) -> int:
    """ Find GCD by recursing on a mod b """
    if(b == 0):
        return abs(a)

    return gcd_recursion(b, a % b)

def gcd_euclidian(a: int, b: int) -> int:
    """ Euler algo """

    # a = q * b - r
    q = a // b
    r = a - q * b
    
    # Stopping condition: r = 0
    if r == 0:
        return b
    
    # next iteration b = r * q_2 + r
    return gcd_euclidian(b, r)

def gcd(a: int, b: int, fn = None) -> int:
    """  """
    # ensure a >= b
    a, b = (a, b) if a >= b else (b, a)

    fn = fn or gcd_euclidian
    return fn(a, b)

def extended_euclidian(a: int, b: int, state:Tuple[int, int, int] = None) -> int:
    """ Euler algo: find gcd(a, b)  """

    # ensure a >= b
    a, b = (a, b) if a >= b else (b, a)
    if state == None:
        state = (np.array([a, 1, 0]), np.array([b, 0, 1]))
    
    # Stopping condition: r = 0
    if state[1][0] <= 0:
        return tuple(v for v in state[0])
    
    # a = q * b - r
    q = state[0][0] // state[1][0]
    w = state[0] - q * state[1]
    state = (state[1], w)
    
    # next iteration b = r * q_2 + r
    return extended_euclidian(a, b, state)

def sieve_of_eratosthenes(n: int) -> Tuple[int,...]:
    """ Simple sieve to find primes up to n """
    n = n if n > 0 else -n
    if n < 2:
        return tuple()

    is_prime_sieve = np.full(n + 1, True)
    is_prime_sieve[0:2] = False
    is_prime_sieve[4::2] = False
    
    # Start with 2 and odd numbers from 3 to n
    sqrt_n = math.ceil(np.sqrt(n))
    for si in range(3, sqrt_n + 1, 2):
        if is_prime_sieve[si]:
            # Mark every multiple of si from si^2
            is_prime_sieve[si ** 2::si] = False
    return tuple(int(i) for i in np.flatnonzero(is_prime_sieve))

def primes_below(n: int):
    """ """
    is_prime = [True, ] * (n + 1)
    for p in range(2, n):
        if is_prime[p]:
            for i in range(p * p, n + 1, p):
                is_prime[i] = False
            yield p
    return

def next_prime(n: int):
    """ """
    for p in primes_below((int(np.ceil(np.sqrt(n))) + 1) ** 2):
        if p > n:
            break
    return p

def is_prime(n: int) -> bool:
    """ check if is a prime by """
    for p in primes_below((int(np.ceil(np.sqrt(n))) + 1) ** 2):
        if p == n:
            return True
    return False

def ifactors(n: int):
    """   """
    n = n if n > 0 else -n
    half_n = n // 2
    factor_list = []
    for p in sieve_of_eratosthenes(half_n):
        if n % p == 0:
            k = 1
            m = n // p
            while m % p == 0:
                m = m // p
                k += 1
            factor_list.append((p, k))
    
    return factor_list or [(n, 1)]

def divisors(n: int) -> Tuple[int, ...]:
    """ """
    factors = tuple(ifactors(n))
    factor_primes = np.array([f[0] for f in factors] + [0,])
    # add one to every power, as we generate powers as (i mod factor_powers[j])
    factor_powers = np.array([f[1] for f in factors] + [0,]) + 1
    
    factors_count = len(factors)
    divisors_count = np.prod(factor_powers)

    # calc product of array of each prime factor to some power, varying from 0 to the max given from ifactors fn
    ds = sorted(int(np.prod([factor_primes[j] ** (i // np.prod(factor_powers[j - factors_count:]) % factor_powers[j])
                             for j in range(factors_count)]))
                for i in range(divisors_count))
    return tuple(ds)

def co_primes(n: int):
    """ """
    return set([a for a in range(1, n) if gcd(a, n) == 1])

def totient(n: int) -> int:
    """ Euler's phi function is the number of values less than a that are co-prime with n """
    return len(co_primes(n))

def order_of_powers(g: int, n: int) -> List[int]:
    """ g ^ k % n for k being co-prime with phi(n) """

    # order_of_powers = sorted(set([pow(g, k, n) for k in co_primes(totient(n))]))
    # keep all calcs mod n to remove overflow errors
    ks = co_primes(totient(n))
    order_of_powers = set()
    g_k = 1
    for k in range(1, n):
        g_k = g_k * g  % n
        if k in ks:
            order_of_powers.add(g_k)
    return sorted(order_of_powers)

def order(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN

    a_k = 1
    for k in range(1, n):
        a_k = a_k * a  % n
        if a_k == 1:
            return k
    
    return np.NaN

def is_order_n(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN
    
    ord_n = totient(n)
    # we can do better than all k < n by only looking at divisors of totient(n)
    phi_n_divisors = divisors(ord_n)
    for k in phi_n_divisors:
        if pow(a, k, n) == 1:
            return k == ord_n
    
    return np.NaN

def cyclic_group(a, n, op):
    group = set([pow(a, k, n) for k in range(1, n)])
    return group

def primitive_roots(n):
    # g is a primitive root modulo n if for every integer a coprime to n, there is some integer k for which gk ≡ a (mod n)
    
    # check n is form 2, 4, p^s, 2p^s, where s is any positive integer and p is an odd prime
    factors = [f for f in ifactors(n)]
    if any((len(factors) < 1 or 2 < len(factors),  
            (len(factors) == 2 and (factors[0][0] != 2 or factors[0][1] > 1)),
            (len(factors) == 1 and factors[0][0] == 2 and factors[0][1] > 2),
            (len(factors) == 1 and factors[0][0] < 2))):
        return [] # Exception("No primitive roots exist")
    
    # find smallest  root
    ord_n = totient(n)
    g = None
    for a in co_primes(n):
        if order(a, n) == ord_n:
            g = a
            break
    
    # There are phi(phi(n)) roots: return all roots using factors co-prime with phi(n)
    prime_roots = order_of_powers(g, n)
     
    assert len(prime_roots) == totient(ord_n)
    assert all(pow(g, ord_n, n) == 1 for g in prime_roots)
    return prime_roots

## Q1.

In [ ]:
lower_bound = 2300
upper_bound = 2600

In [ ]:
subset_a = set(p for p in sieve_of_eratosthenes(upper_bound) if p >= lower_bound)
subset_b = set(n for n in range(lower_bound, upper_bound+1)) - subset_a

a)	List the elements of the set A = {all primes p in the range}, B = {all composite numbers in the range}

In [ ]:
print(subset_a)
print(f"In the range [{lower_bound}, {upper_bound}] there are {len(subset_a)} primes. \t# Check with is_prime: {sum(1 for n in subset_a if is_prime(n))}")
print(subset_b)
print(f"In the range [{lower_bound}, {upper_bound}] there are {len(subset_b)} composites.\t# Check with is_prime: {sum(1 for n in subset_b if is_prime(n))}")

b)	List the elements of the set C where C = {composite numbers n = pq in your range which are the product of exactly two distinct primes p and q}.

In [ ]:
prime_divisor_count = lambda n_factors: len(n_factors)
is_square_free = lambda n_factors: len([pf for pf in n_factors if pf[1] > 1]) == 0
subset_c = [c[0] for c in [(b, ifactors(b)) for b in subset_b] if prime_divisor_count(c[1]) == 2 and is_square_free(c[1])]
print(f"In the range [{lower_bound}, {upper_bound}] there are {len(subset_c)} composites of the form n=pq.");

c) Choose any three element of the set B and then randomly select 5 values of a for each element. Apply the gcd test for each of the 12 cases and report on how accurate it is in determining that a number is composite.  

In [ ]:
#is_prime_gcd_test := m -> evalb(igcd(m, rand(2 .. m - 1)()) = 1);
#bs := seq(B[rand(1 .. numelems(B))()], i = 1 .. 3);
#is_prime_results := [seq([seq(is_prime_gcd_test(b), i = 1 .. 5)], b in bs)];
#test_accuracies := [seq(add(subs([true = 0, false = 1], result))/numelems(result), result in is_prime_results)];
#printf("Success for testing that a composite is not prime by gcd, for %d runs against %d samples, is %.2f%%\n", nops(prime_tests), nops(op(1, prime_tests)), 100*Mean(test_accuracies));

Question 2 (10 marks): 

Find all Carmichael Numbers in your range using:

a)	A direct method employing the Fermat Test that shows that a composite number n has no Fermat Witnesses;

In [ ]:
#fermat_tests := m -> local a; [seq(evalb(a^(m - 1) mod m <> 1), a in select(a -> igcd(a, m) = 1, [seq(a, a = 2 .. m - 1)]))];
#fermat_witnesses := m -> add(subs(true = 1, false = 0, fermat_tests(m)));
#fermat_liars := m -> add(subs(true = 0, false = 1, fermat_tests(m)));
#cns := select(b -> fermat_witnesses(b) = 0, [seq(b, b in B)]);
#`~`[cn -> [fermat_witnesses(cn), fermat_liars(cn)]](cns);

b)            Checking which numbers satisfy Korselt’s Criteria. 

In [ ]:
#is_square_free := n -> local pf; `and`(seq(evalb(op(2, pf) = 1), pf in op(2, ifactors(n))));
#prime_divisors := n -> local pf; seq(op(1, pf), pf in op(2, ifactors(n)));
#has_korselt_criteria := n -> local p; `and`(is_square_free(n), seq(evalb(irem(n - 1, p - 1) = 0), p in prime_divisors(n)));
#cns := select(b -> has_korselt_criteria(b), [seq(b, b in B)]);

Question 3 (25 marks):

Take the first five elements n of the set B of composite numbers with 2 factors in your range (or all numbers if you find there are less than 10). 

The Miller Rabin test states that at most ¼ of numbers a that are randomly chosen will give the answer that n is ‘probably prime’. How close can you get to this maximum, (i.e. which of your 5 choices has the highest proportion of possible a’s that would fail the Miller Rabin test).

In [ ]:
#is_odd := n -> evalb(irem(n, 2) = 1);
#powers_of_2 := n -> if is_odd(n) then return 0; else return op(2, op(1, op(2, ifactors(n)))); end if;
#is_miller_rabin_prime := proc(m, a) local s, i, d, x, y; s := powers_of_2(m - 1); d := (m - 1)/2^s; x := a^d mod m; for i to s do y := x^2 mod m; if y = 1 and x <> 1 and x <> m - 1 then return false; end if; x := y; end do; return evalb(y = 1); end proc;

In [ ]:
#C := select(b -> `and`(prime_divisor_count(b) = 2, true), B);
#C :# select(b -> `and`(prime_divisor_count(b) = 2, is_odd(b)), B);
#seq([n, ifactor(n), add(subs(true = 1, false = 0, [seq(is_miller_rabin_prime(n, a), a = 2 .. n - 2)]))/(n - 3)], n in [seq(C[i], i = 1 .. 5)]);
#select(mr_result -> 2 < op(1, op(3, mr_result)), [seq([n, ifactor(n), add(subs(true = 1, false = 0, [seq(is_miller_rabin_prime(n, a), a = 2 .. n - 2)]))/(n - 3)], n in B)]);

What composite numbers m between 50 and 100 have the highest proportion of Miller Rabin failures? (For each number in the range work out the proportion of a’s that produce the answer ‘m is probably prime’). Look at the prime factorisation of these numbers and see if it suggests any patterns about which numbers are vulnerable to giving false answers in Miller Rabin.  

In [ ]:
#select(mr_result -> 0 < op(3, mr_result) and is_odd(op(1, mr_result)), [seq([n, ifactor(n), add(subs(true = 1, false = 0, [seq(is_miller_rabin_prime(n, a), a = 2 .. n - 2)]))/(n - 3)], n in select(n -> not isprime(n), [seq(i, i = 50 .. 100)]))]);
#select(mr_result -> op(3, mr_result) = 0 and is_odd(op(1, mr_result)), [seq([n, ifactor(n), add(subs(true = 1, false = 0, [seq(is_miller_rabin_prime(n, a), a = 2 .. n - 2)]))/(n - 3)], n in select(n -> not isprime(n), [seq(i, i = 50 .. 100)]))]);

Question 4 (15 marks):

a)	Choose any three elements of your set A and calculate the value of r used in the AKS primality test;

b)	Write a single procedure that implements the AKS test using the code that we have seen;

c)	Take the elements of the set B in turn and decide how many fail the test at each of steps 1, 2, 3, 4, 5

Question 6 (contributes to the ‘written explanation’ category worth 25% of the marks, you should write 400-500 words summarising your conclusions):

Consider the tests we have seen so far in the module

		i)	a Fermat Test calculating am-1 mod m
        
		ii)	a gcd test on a and m
        
		iii)	a Miller Rabin test 
        
		iv) 	Trial division/sieving methods
        
		v) 	The AKS primality test
        
Thinking about factors such as:

•	the probability that the test produces a clear answer, 

•	the amount of work that it involves 

summarise which test you would recommend for deciding if a number is prime or not.  

Does the size of the target number affect your answer? Does it change for:

a)	Numbers less than 10 000 000

b)	Numbers bigger than 1 000 000 000 000

c)	Numbers bigger than 10^100